# Connect Neo4J to NetworkX
The following gives you a connection to your neo4J DB including the "Network of Thrones" data set. Please make sure everything is set up as described in the main README.

In [1]:
from neo4j import GraphDatabase
import networkx as nx

# Replace with your credentials. Default credentials from main README.
uri = "bolt://localhost:7687"
username = "neo4j"
password = "your_password"

with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    driver.verify_connectivity()

def run_query(query, parameters=None):
        """ Run a Cypher query with optional parameters. """
        with driver.session() as session:
            result = session.run(query, parameters)
            return result.data()  # Return query results as a list of dictionaries

This gives you the overall count of movie characters included in all of the 5 books.

In [2]:
query = "MATCH (c:Character) RETURN count(c) AS count"
result = run_query(query)
count = result[0]["count"] if result else 0
print(f"There are {count} characters in the database.")

There are 796 characters in the database.


/var/folders/w4/ql5lg1bd7rb18j2pdg75j9v40000gn/T/ipykernel_27608/1238049016.py:14: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


## Task 1.1: Count Interactions

Could you create a query that counts the total number of interactions and stores it in the variable interaction_count

In [ ]:
query = #TODO
result = run_query(query)
interaction_count = result[0]["interaction_count"] if result else 0
print(f"There are {interaction_count} interactions in the database.")

## Task 1.2: Count Interactions by Book

Can you come up with a query that counts the interactions ordered by book?

In [ ]:
query = #TODO
result = run_query(query)

for book in result:
    print(f"Book {book['book']} has {book['interaction_count']} interactions.")

## Task 1.3: Load the entire Graph to NetworkX

Please complete the missing code in the following cell to load the entire graph into a networkX graph.

In [ ]:
query = """
        MATCH (n)
        WITH collect(n) AS nodes
        MATCH ()-[r]->()
        RETURN nodes, collect(r) AS relationships
        """

# Run the query    
result = run_query(query)

# Create a new NetworkX graph
graph = #TODO

# Add nodes to the graph
for record in result:
   for node in record["nodes"]:
      #TODO

   # Add relationships to the graph
   for rel in record["relationships"]:  
      #TODO
      
# Print the graph's nodes and edges
num_edges = #TODO
num_nodes = #TODO
print(f"The graph has {num_nodes} nodes and {num_edges} edges")